In [ ]:
from srai.regionalizers import geocode_to_region_gdf

prague_area = geocode_to_region_gdf('Praha, CZ')
prague_area.explore()

In [ ]:
import osmnx as ox

def map_flats(flats_value: str) -> int:
    try:
        flats = int(flats_value)
    except:
        flats = 1

    return flats

residential_buildings = ox.geometries_from_place(query='Praha, CZ', tags={"building": "residential"})

residential_buildings['flats'] = residential_buildings['building:flats'].apply(map_flats)
residential_buildings = residential_buildings.reset_index()[['osmid', 'geometry', 'flats']]
residential_buildings.geometry = residential_buildings.geometry.apply(lambda geometry: geometry.centroid)

residential_buildings

In [ ]:
import geopandas as gpd

cadastral_data = gpd.read_file('data/cadastral_data.geojson')
cadastral_data.explore(column='population', tiles="CartoDB positron", style_kwds=dict(opacity=0))

In [ ]:
from tqdm import tqdm

# bsu - basic settlement unit
for bsu in tqdm(cadastral_data.to_dict(orient='records')):
    matching_buildings = residential_buildings[residential_buildings.intersects(bsu['geometry'])]
    total_flats = matching_buildings['flats'].sum()
    for building_index, building_row in matching_buildings.iterrows():
        residential_buildings.loc[building_index, 'population'] = bsu['population'] * (building_row['flats'] / total_flats)
        
residential_buildings.plot('population')

In [ ]:
fast_foods  = ox.geometries_from_place(query='Praha, CZ', tags={"amenity": "fast_food"})
fast_foods = fast_foods.reset_index()[['osmid', 'geometry', 'brand']]
fast_foods = fast_foods[fast_foods.brand.isin(['KFC', "McDonald's"])]
fast_foods.geometry = fast_foods.geometry.apply(lambda geometry: geometry.centroid)
fast_foods.set_index('osmid', inplace=True)
fast_foods

In [ ]:
fast_foods.brand.value_counts()

In [ ]:
from srai.regionalizers import VoronoiRegionalizer

voronoi_regions = VoronoiRegionalizer(seeds=fast_foods).transform(gdf=prague_area)
voronoi_regions

In [ ]:
population_in_regions = voronoi_regions.sjoin(residential_buildings).groupby('region_id')['population'].sum()
population_in_regions

In [ ]:
regions_with_population = voronoi_regions.join(fast_foods[['brand']]).join(population_in_regions)
regions_with_population

In [ ]:
brand_closest_population = voronoi_regions.groupby('brand').agg({'population': 'sum'}).reset_index()
brand_closest_population['percentage'] = 100 * brand_closest_population['population'] / brand_closest_population['population'].sum()
brand_closest_population

In [ ]:
from srai.plotting.folium_wrapper import _generate_linear_colormap

brand_color_mapping = {"KFC": ("#fa9ea0", "#a3080c"), "McDonald's": ("#ffeec0", "#ffc72c")}

m = None

for brand, colors in brand_color_mapping.items():
    regions_subset = voronoi_regions[voronoi_regions["brand"] == brand]
    m = regions_subset.explore(
        m=m,
        column="population",
        cmap=_generate_linear_colormap(
            colors,
            min_value=0,
            max_value=regions_subset["population"].max(),
        ),
        tiles="CartoDB positron",
        style_kwds=dict(opacity=0.25, color=colors[1]),
    )

fast_foods.explore(
    m=m,
    style_kwds=dict(color="#444", opacity=1, fillColor="#f2f2f2", fillOpacity=1),
    marker_kwds=dict(radius=3),
)